# Train LSTM Model Manually

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
from torch import nn

from models import HarLSTM, ModelUtils
from pl_data import HarDataModule
from utils import FeatUtils

%load_ext autoreload
%autoreload 2

# 1. Prepare features

In [ ]:
data_dir_path = "./data/har_dataset"
norm_method = "std"
batch_size = 16
data_module = HarDataModule(data_dir_path, 
                            batch_size=batch_size,
                           normalize=norm_method)

# 2. Define model structure and parameters

In [ ]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
if(use_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

In [ ]:
# Instantiate the model with hyperparams
input_size = 9
output_size = 6
n_hidden = 128
n_layers = 2

# Training params
epochs = 50
lr = 0.0001

net = HarLSTM(input_size, output_size, n_hidden=n_hidden, n_layers=n_layers)

print("Model information:")
print(net)

# Define loss and optimization functions
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# 3. Train the model

In [ ]:
train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
train_stat_dict = ModelUtils.train_net(net, criterion, optimizer, train_loader, val_loader, batch_size, epochs, 
                            use_gpu=use_gpu, print_every=100, clip=5)

In [ ]:
ModelUtils.plot_loss_chart(train_stat_dict)

In [ ]:
# Save a model
model_path = f"har_lstm_{batch_size}_ep{epochs}_{norm_method}.pt"
torch.save(net, model_path)

# 4. Test inference by a loaded model

In [ ]:
loaded_net = torch.load(model_path, map_location="cpu")

In [ ]:
test_loader = data_module.test_dataloader()
test_loss, test_labels, preds = ModelUtils.test_net(loaded_net, loaded_net.criterion, test_loader, batch_size, use_gpu=use_gpu)

In [ ]:
acc = accuracy_score(test_labels, preds)
prec, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average="macro")

In [ ]:
print(f"accuracy: {acc}")
print(f"precision: {prec}")
print(f"recall: {recall}")
print(f"f1: {f1}")